# 08 ODE integrators: Verlet (Students)

In [ ]:
from importlib import reload

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

import integrators

In [ ]:
reload(integrators)

## Velocity Verlet

Use expansion *forward* and *backward* (!) in time (Hamiltons (i.e. Newton without friction) equations are time symmetric)

\begin{align}
r(t + \Delta r) &\approx r(t) + \Delta t\, v(t) + \frac{1}{2m} \Delta t^2 F(t)\\
r(t) &\approx r(t + \Delta t) - \Delta t\, v(t + \Delta t) + \frac{1}{2m} \Delta t^2 F(t+\Delta t)
\end{align}

Solve for $v$:
\begin{align}
v(t+\Delta t) &\approx v(t) + \frac{1}{2m} \Delta t \big(F(t) + F(t+\Delta t)\big)
\end{align}

Complete **Velocity Verlet** integrator consists of the first and third equation.

In practice, split into three steps (calculate the velocity at the half time step):
\begin{align}
v(t+\frac{\Delta t}{2}) &= v(t) + \frac{\Delta t}{2} \frac{F(t)}{m} \\
r(t + \Delta r) &= r(t) + \Delta t\, v(t+\frac{\Delta t}{2})\\
v(t+\Delta t) &= v(t+\frac{\Delta t}{2}) + \frac{\Delta t}{2} \frac{F(t+\Delta t)}{m}
\end{align}

When writing production-level code, remember to re-use $F(t+\Delta t)$ als the "new" starting $F(t)$ in the next iteration (and don't recompute).

### Integration of planetary motion 
\begin{align}
\mathbf{F} &= -\frac{G m M}{r^2} \hat{\mathbf{r}}\\
\hat{\mathbf{r}} &= \frac{1}{\sqrt{x^2 + y^2}} \left(\begin{array}{c} x \\ y \end{array}\right)
\end{align}
Set $$GM = 1$$ and try initial conditions
$$
x(0) = 0.5, \quad y(0)=0, \quad v_x(0)=0, \quad v_y(0)=1.63
$$

In [ ]:
def F_gravity(r, m=1, G=1, M=1):
    # implement

In [ ]:
# 2D planetary motion with velocity verlet
dim = 2

r0 = np.array([0.5, 0])
v0 = np.array([0, 1.63])
mass = 1

dt = 0.01
t_max = 100
nsteps = int(t_max/dt)

# implement

In [ ]:
rx, ry = r.T
plt.plot(rx, ry)

### Velocity Verlet toy systems

In [ ]:
t, y = integrators.integrate_newton(0, 1, 2000, h=0.01, 
                             force=integrators.F_harmonic,
                             integrator=integrators.velocity_verlet)

In [ ]:
ax = plt.subplot(1,1,1)
ax.plot(t, y[:,0])
ax.set_xlim(1000, 1020)

In [ ]:
x, v = y.T
plt.plot(x, v)

In [ ]:
integrators.analyze_energies(t, y, integrators.U_harmonic, step=100)

In [ ]:
integrators.energy_conservation(t, y, integrators.U_harmonic)

In [ ]:
t_long, y_long = integrators.integrate_newton(0, 1, 50000, h=0.01, 
                             force=integrators.F_harmonic,
                            integrator=integrators.velocity_verlet)

In [ ]:
integrators.energy_conservation(t_long, y_long, integrators.U_harmonic)

In [ ]:
%%timeit 
t, y = integrators.integrate_newton(0, 1, 200, h=0.01, 
                             force=integrators.F_harmonic,
                            integrator=integrators.velocity_verlet)

## RK4 

In [ ]:
t4, y4 = integrators.integrate_newton(0, 1, 2000, h=0.01, 
                             force=integrators.F_harmonic,
                            integrator=integrators.rk4)

In [ ]:
integrators.analyze_energies(t4, y4, integrators.U_harmonic, step=100)

In [ ]:
integrators.energy_conservation(t4, y4, integrators.U_harmonic)

In [ ]:
%%timeit
t4, y4 = integrators.integrate_newton(0, 1, 200, h=0.01, 
                             force=integrators.F_harmonic,
                            integrator=integrators.rk4)

In [ ]:
t4_long, y4_long = integrators.integrate_newton(0, 1, 50000, h=0.01, 
                             force=integrators.F_harmonic,
                            integrator=integrators.rk4)

In [ ]:
integrators.energy_conservation(t4_long, y4_long, integrators.U_harmonic)

# Gravitational orbits

\begin{align}
\mathbf{F} &= -\frac{G m M}{r^2} \hat{\mathbf{r}}\\
\hat{\mathbf{r}} &= \frac{1}{\sqrt{x^2 + y^2}} \left(\begin{array}{c} x \\ y \end{array}\right)
\end{align}
Set $$GM = 1$$ and try initial conditions
$$
x(0) = 0.5, \quad y(0)=0, \quad v_x(0)=0, \quad v_y(0)=1.63
$$

In [ ]:
def F_gravity(r, m=1, G=1, M=1):
    rr = np.sum(r*r)
    rhat = r/np.sqrt(rr)
    return - G*m*M/rr * rhat

In [ ]:
r0 = np.array([0.5, 0])
v0 = np.array([0, 1.63])

In [ ]:
F_gravity(r0)

Use the new function `integrators.integrate_newton_2d()` to integrate 2d coordinates.

In [ ]:
t, y = integrators.integrate_newton_2d(x0=r0, v0=v0, t_max=10, mass=1,
                            force=F_gravity, 
                            integrator=integrators.rk4)

In [ ]:
rx, ry = y[:, 0].T

In [ ]:
plt.plot(rx, ry)

In [ ]:
te, ye = integrators.integrate_newton_2d(x0=r0, v0=v0, t_max=10, mass=1,
                            force=F_gravity, 
                            integrator=integrators.euler)
rex, rey = ye[:, 0].T

In [ ]:
ax = plt.subplot(1,1,1)
ax.plot(rx, ry, label="RK4")
ax.plot(rex, rey, label="Euler")
ax.legend(loc="best")